In [9]:
import wrds
import pandas as pd
import numpy as np
import requests
import json
import os
import tqdm
import pandas_datareader.data as reader

## WRDS Connection

In [17]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
Loading library list...
Done


## Tickers
Aggregation of all current US stock symbols from NASDAQ, NYSE, AMEX

In [18]:
# read in all_tickers.txt
with open('all_tickers.txt', 'r') as f:
    tickers = f.readlines()
tickers = [x.strip() for x in tickers]

tickers_compiled = tuple(tickers)
#len(tickers_tuple)
#tickers_tuple


## Params
Change the following for automatic data queries.

In [19]:
# CHANGE ME
# DATE RANGE
start_date = ('2015-01-01')
end_date = ('2023-03-01')

In [20]:
# DON'T CHANGE ME
params = {'tickers': tickers_compiled, 'start_date': start_date, 'end_date': end_date}

In [21]:
tables = db.list_tables("wrdsapps")
# check whether the table contains m_stock
#print(db.describe_table(library='wrdsapps_finratio', table='firm_ratio'))
#print(tables)
ratio_table = db.describe_table(library='wrdsapps_finratio', table='firm_ratio')

pd.set_option('display.max_rows', 98)
#ratio_table
# print out the value 
# ratio_table

Approximately 2782963 rows in wrdsapps_finratio.firm_ratio.


## Financial Ratios

In [22]:
# Financial Ratios: change the first line in SELECT ...... to include more ratios
# start date and end date can be changed above

data_ratios = db.raw_sql(
    """
    SELECT PERMNO as PERMNO, gvkey as gvkey,TICKER, public_date as date, bm as book_to_market, 
    debt_assets, pe_inc as price_to_earnings, Accrual     
    FROM wrdsapps_finratio.firm_ratio 
    WHERE TICKER IN %(tickers)s AND public_date >= %(start_date)s AND public_date <= %(end_date)s
    """,
params=params)

In [23]:
data_ratios.shape

(256528, 8)

## Firm Info, Price, and More

In [24]:
# company monthly stock data
data_price = db.raw_sql(
    """
    SELECT PERMNO as PERMNO,  Ticker as Ticker, MthCalDt as Date, ICBIndustry as ICB_industry_code, MthRet as monthly_return, sprtrn as snp_monthly_return, MthCap as monthly_mktcap, MthPrcVol as monthly_price_volume, ShrOut as shares_outstanding, MthPrc as monthly_price
    FROM crspm.wrds_msfv2_query
    WHERE Ticker in %(tickers)s AND MthCalDt >= %(start_date)s AND MthCalDt <= %(end_date)s
    """, 
    params=params)

In [25]:
data_price.shape

(439255, 10)

## Bonds (Rf)

In [26]:
# Bonds
data_bond = db.raw_sql(
    """
    SELECT caldt as date, b10ret as ten_yr_YTM_rf
    FROM crspm.mcti
    WHERE caldt >= %(start_date)s AND caldt <= %(end_date)s
    """,
    params=params)

## ESG Scores
Unfortunately, WRDS does not support queries directly via SQL, so the data was obtained through the WRDS query forms 

In [27]:
# ESG Data Processing
# read in ESG data
esg_score_raw = pd.read_excel('snpESG_all_ex.xlsx')


In [28]:
# get rid of the .0 in gvkey
esg_score_raw['gvkey'] = esg_score_raw['gvkey'].astype(str).str[:-2]

In [29]:
esg_score_raw

,institutionid,Score Date,Aspect Name,Score Value,gvkey,Ticker,Company Name,Incorporation State,State,Zip Code
0,100003,2017-04-19,S&P Global ESG Score,34,24447,BOKF,BOK Financial Corporation,OK,OK,74172
1,100003,2017-04-19,Economic Governance Dimension,50,24447,BOKF,BOK Financial Corporation,OK,OK,74172
2,100003,2017-04-19,Environmental Dimension,23,24447,BOKF,BOK Financial Corporation,OK,OK,74172
3,100003,2017-04-19,Social Dimension,21,24447,BOKF,BOK Financial Corporation,OK,OK,74172
4,100003,2017-04-19,S&P Global ESG Score,23,24447,BOKF,BOK Financial Corporation,OK,OK,74172
...,...,...,...,...,...,...,...,...,...,...
166563,109851650,2023-02-17,Social Dimension,11,38015,HLGN,"Heliogen, Inc.",DE,CA,91103
166564,109851650,2023-02-17,S&P Global ESG Score,8,38015,HLGN,"Heliogen, Inc.",DE,CA,91103
166565,109851650,2023-02-17,Economic Governance Dimension,20,38015,HLGN,"Heliogen, Inc.",DE,CA,91103
166566,109851650,2023-02-17,Environmental Dimension,0,38015,HLGN,"Heliogen, Inc.",DE,CA,91103


In [30]:
# re-arrange so that company id goes first, and ticker follows, the Score Data, Aspect Name, Socre Value, and institutionid
esg_score_raw = esg_score_raw[['gvkey', 'Company Name', 'Ticker', 'Score Date', 'Aspect Name', 'Score Value', 'institutionid']]
# Movie values of aspect name to columns
esg_score_raw = esg_score_raw.pivot_table(index=['gvkey','Company Name', 'Ticker', 'Score Date', 'institutionid'], columns='Aspect Name', values='Score Value')
# sort by score date
esg_score_raw = esg_score_raw.sort_values(by=['Score Date'])

In [31]:
esg_score_raw = esg_score_raw.sort_values(by=['Score Date'])
esg_score_finished = esg_score_raw.reset_index()
#esg_score_finished.head()

In [32]:
# check whether there are duplicates in esg_score_finished
esg_score_finished.duplicated().sum()
# for fama-french
#fama_french_betas.duplicated().sum()

0

In [33]:
# rename Ticker to ticker, Score Date to date
esg_score_finished = esg_score_finished.rename(columns={'Ticker': 'ticker', 'Score Date': 'date'})
#esg_score_finished.head()

In [34]:
# merge ESG data with financial ratios based on ticker and date
# change the date column in data_price and data_ratios to date format
data_price['date'] = pd.to_datetime(data_price['date'])
data_ratios['date'] = pd.to_datetime(data_ratios['date'])

# merge data_price and data_ratios
data_price_ratios = pd.merge(data_price, data_ratios, on=['ticker', 'date', 'permno' ], how='inner')

In [35]:
# read in betas
fama_french_betas = pd.read_excel('fama_french_betas.xlsx')
capm_betas = pd.read_excel('capm_beta.xlsx')

In [36]:
data_bond['date'] = pd.to_datetime(data_bond['date'])
data_price_ratios['date'] = pd.to_datetime(data_price_ratios['date'])
esg_score_finished['date'] = pd.to_datetime(esg_score_finished['date'])
capm_betas['date'] = pd.to_datetime(capm_betas['date'])
fama_french_betas['date'] = pd.to_datetime(fama_french_betas['date'])
# in data_ratios, calculate shares_turnover, which is monthly_price_volume/shares_outstanding
data_price['shares_turnover'] = data_price['monthly_price_volume']/data_price['shares_outstanding']

In [37]:
# rename fama-french Ticker Symbol to ticker
fama_french_betas = fama_french_betas.rename(columns={'Ticker Symbol': 'ticker'})

In [38]:
# merge data_price_ratios and esg_score_finished
# change the date column in data_price_ratios and esg_score_finished to year-month-day format
data_price_ratios['date'] = data_price_ratios['date'].dt.strftime('%Y-%m')
esg_score_finished['date'] = esg_score_finished['date'].dt.strftime('%Y-%m')
data_bond['date'] = data_bond['date'].dt.strftime('%Y-%m')
capm_betas['date'] = capm_betas['date'].dt.strftime('%Y-%m')
fama_french_betas['date'] = fama_french_betas['date'].dt.strftime('%Y-%m')



In [39]:
type(esg_score_finished['gvkey'][0])
# change the type of gvkey in esg_score_finished to int
esg_score_finished['gvkey'] = esg_score_finished['gvkey'].astype(str)
# rename fama_french_betas PERMNO to permno
fama_french_betas = fama_french_betas.rename(columns={'PERMNO': 'permno'})
capm_betas = capm_betas.rename(columns={'PERMNO': 'permno'})

## Merging the dataframes

In [40]:
# merge data_price_ratios and esg_score_finished
data_incomplete = pd.merge(data_price_ratios, esg_score_finished, on=['ticker', 'date','gvkey'], how='right')
# drop company name and institutionid from data_incomplete
data_incomplete = data_incomplete.drop(['Company Name', 'institutionid'], axis=1)
data_complete = pd.merge(data_incomplete, data_bond, on=['date'], how='left')
# merge data_complete_missing_beta and fama_french_betas
# data_complete_missing_capm_beta = pd.merge(data_complete_missing_beta, fama_french_betas, on=['ticker', 'date', 'permno'], how='inner')
# merge data_complete_missing_beta and capm_betas
#data_complete = pd.merge(data_complete_missing_capm_beta, capm_betas, on=['ticker', 'date', 'permno'], how='inner')

In [41]:
# print the columns with the most NAs
data_complete.isna().sum().sort_values(ascending=False).head(20)

book_to_market                   17391
price_to_earnings                17314
accrual                          17308
debt_assets                      17307
permno                           17302
icb_industry_code                17302
monthly_return                   17302
snp_monthly_return               17302
monthly_mktcap                   17302
monthly_price_volume             17302
shares_outstanding               17302
monthly_price                    17302
ten_yr_ytm_rf                     1405
ticker                               0
date                                 0
Economic Governance Dimension        0
Environmental Dimension              0
S&P Global ESG Score                 0
Social Dimension                     0
gvkey                                0
dtype: int64

In [42]:
# drop NA values
data_complete = data_complete.dropna()

In [43]:
data_price_ratios.groupby('date')['permno'].nunique()

date
2015-03    2014
2015-04    2014
2015-06    2034
2015-07    2035
2015-08    2070
2015-09    2071
2015-11    2107
2015-12    2106
2016-02    2128
2016-03    2129
2016-05    2140
2016-06    2140
2016-08    2170
2016-09    2170
2016-10    2170
2016-11    2200
2017-01    2197
2017-02    2228
2017-03    2228
2017-05    2255
2017-06    2257
2017-07    2257
2017-08    2302
2017-10    2301
2017-11    2331
2018-01    2332
2018-02    2380
2018-04    2379
2018-05    2423
2018-07    2428
2018-08    2475
2018-10    2480
2018-11    2528
2018-12    2532
2019-01    2532
2019-02    2574
2019-04    2573
2019-05    2602
2019-07    2604
2019-09    2669
2019-10    2669
2019-12    2717
2020-01    2718
2020-03    2764
2020-04    2764
2020-06    2797
2020-07    2795
2020-08    2844
2020-09    2846
2020-11    2915
2020-12    2921
2021-03    3028
2021-04    3029
2021-06    3142
2021-08    3284
2021-09    3291
2021-11    3464
2021-12    3472
2022-01    3473
2022-02    3613
2022-03    3619
2022-05    3686
202

In [44]:
data_complete.columns

Index(['permno', 'ticker', 'date', 'icb_industry_code', 'monthly_return',
       'snp_monthly_return', 'monthly_mktcap', 'monthly_price_volume',
       'shares_outstanding', 'monthly_price', 'gvkey', 'book_to_market',
       'debt_assets', 'price_to_earnings', 'accrual',
       'Economic Governance Dimension', 'Environmental Dimension',
       'S&P Global ESG Score', 'Social Dimension', 'ten_yr_ytm_rf'],
      dtype='object')

In [45]:
# rename 'Economic Governance Dimension', 'Environmental Dimension' 'S&P Global ESG Score', 'Social Dimension' to be without spaces and lowercase
data_complete = data_complete.rename(columns={'Economic Governance Dimension': 'economic_gov_dim', 'Environmental Dimension': 'env_dim', 'S&P Global ESG Score': 'sp_esg_score', 'Social Dimension': 'social_dim'})

#data_complete.columns

## Output Stocks Data

In [46]:
# export to excel
#data_ratios.to_excel('financial_ratios.xlsx', index=False)
#esg_score_finished.to_excel('ESG_Score_Processed.xlsx', index=False)
data_complete.to_excel('data_complete_2.xlsx', index=False)
#data_complete_2021_11.to_excel('data_complete_2021_11.xlsx', index=False)

## Fama-French Factors

In [76]:
# Index by zero here since we only need the first df (monthly instead of yearly)
factors = reader.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start_date, end=end_date)[0]

In [82]:
# some cleaning to prepare for merging
factors = factors.reset_index(inplace=False)
factors.rename(columns={'Date': 'date'}, inplace=True)
#datetime YYYY-MM
factors['date'] = factors['date'].dt.strftime('%Y-%m')

In [83]:
# Merge factors with data_complete on date
data_complete_ff = pd.merge(data_complete, factors, on='date', how='left')
data_complete_ff

,permno,ticker,date,icb_industry_code,monthly_return,snp_monthly_return,monthly_mktcap,monthly_price_volume,shares_outstanding,monthly_price,...,env_dim,sp_esg_score,social_dim,ten_yr_ytm_rf,level_0,index,Mkt-RF,SMB,HML,RF
0,13318.0,YELP,2015-03,TECH,-0.013542,-0.017396,3.072542e+06,2.692096e+09,64890.0,47.35,...,14.5,29.0,14.0,0.008165,2,2,-1.12,3.04,-0.37,0.00
1,13511.0,PANW,2015-03,TECH,0.027141,-0.017396,1.201844e+07,5.682797e+09,82273.0,146.08,...,14.5,26.5,18.5,0.008165,2,2,-1.12,3.04,-0.37,0.00
2,87056.0,BMRN,2015-03,HEALTH,0.163911,-0.017396,1.997434e+07,5.456174e+09,160282.0,124.62,...,16.5,34.0,26.5,0.008165,2,2,-1.12,3.04,-0.37,0.00
3,92257.0,VMW,2015-03,TECH,-0.035970,-0.017396,1.046587e+07,2.414772e+09,127617.0,82.01,...,22.5,35.0,19.0,0.008165,2,2,-1.12,3.04,-0.37,0.00
4,13447.0,NOW,2015-03,TECH,0.033045,-0.017396,1.168307e+07,1.807558e+09,148300.0,78.78,...,17.0,31.5,16.0,0.008165,2,2,-1.12,3.04,-0.37,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1911,13641.0,FANG,2022-11,ENERGY,-0.044464,0.053753,2.605137e+07,8.698511e+09,175999.0,148.02,...,35.0,39.0,29.0,0.040789,94,94,4.60,-3.40,1.38,0.29
1912,13641.0,FANG,2022-11,ENERGY,-0.044464,0.053753,2.605137e+07,8.698511e+09,175999.0,148.02,...,35.0,39.0,29.0,0.040789,94,94,4.60,-3.40,1.38,0.29
1913,89216.0,AYI,2022-11,INDL,0.025712,0.053753,6.055030e+06,8.014047e+08,32158.0,188.29,...,28.5,31.0,26.5,0.040789,94,94,4.60,-3.40,1.38,0.29
1914,90312.0,WLK,2022-11,BASMAT,0.117553,0.053753,1.372947e+07,1.795030e+09,127538.0,107.65,...,32.0,33.5,30.0,0.040789,94,94,4.60,-3.40,1.38,0.29


In [84]:
data_complete_ff.to_excel('data_complete_ff.xlsx', index=False)

## Old Code (Ignore everything below)
For record keeping purposes

In [ ]:
# DON't mind these data below, just for record keeping
# turn tickers_compiled into a dataframe
#tickers_compiled = pd.DataFrame(tickers_compiled)
# export to txt file
# tickers_compiled.to_excel('tickers_compiled.xlsx')
# VARIABLES 
# variables = ("TICKER", "public_date", "bm", "capital_ratio", "evm", "ptb", "roe")
# LIBRARY
# library = ('wrdsapps_finratio')
# Table
# table = ('firm_ratio')
# date_name
# date_name = ('public_date')
# ticker_name
# ticker_name = ('ticker')
# list tables 
# db.list_tables(library='wrdsapps_finratio')
# Find all the column names in the table
# print(db.describe_table(library='wrdsapps_finratio', table='firm_ratio'))
# Run the following code to see what the schema is for the for any tables in a library
data_test = db.raw_sql(
    """
    SELECT *
    FROM trucost.wrds_esg
    LIMIT 6
    """,
params=params)
data_test.columns()

StatementError: (builtins.AttributeError) 'NoneType' object has no attribute 'cursor'
[SQL: 
    SELECT *
    FROM trucost.wrds_esg
    LIMIT 6
    ]
[parameters: [{'tickers': ('A', 'AACS', 'AAIC', 'AAIIQ', 'AAL', 'AAME', 'AAM PR A', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AASP', 'AAT', 'AATC', 'AATV', 'AAWW', 'A ... (33300 characters truncated) ... 'ZG', 'ZION', 'ZM', 'ZMA', 'ZNOG', 'ZNRG', 'ZRFY', 'ZS', 'ZTNO', 'ZUMZ', 'ZUO', 'ZVOI', 'ZWS'), 'start_date': '2015-01-01', 'end_date': '2023-03-01'}]]

In [ ]:
# read in Final_data_yearly.xls

yearly_data = pd.read_excel('Final_data_yearly.xls')


In [ ]:
yearly_data.head()

,CompanyName,ticker,year,institutionid,EconomicGovernanceDimension,EnvironmentalDimension,SPGlobalESGScore,SocialDimension,PERMNO,gvkey,...,b_mkt_fama_french_3fac1,b_smb_fama_french_3fac1,b_hml_fama_french_3fac1,alpha_fama_french_3fac1,excess_return_fama_french_3fac1,permno,icb_industry_code,mktcap,yearly_return,industry
0,"Agilent Technologies, Inc.",A,2019,4075849,51.0,75.0,53.0,50.0,87432,126554,...,4.321428,-16.811880,-87.152542,0.021524,-2.575978,87432,HEALTH,23936570.0,0.275302,6
1,"Agilent Technologies, Inc.",A,2020,4075849,47.0,80.0,51.0,50.0,87432,126554,...,-0.433493,4.535038,13.811031,0.030218,-0.400570,87432,HEALTH,28794336.0,0.397681,6
2,"Agilent Technologies, Inc.",A,2021,4075849,50.0,77.0,55.0,55.0,87432,126554,...,-7.571819,-9.820291,-2.614265,0.027274,0.522970,87432,HEALTH,43736736.0,0.280805,6
3,American Airlines Group Inc.,AAL,2019,4065191,43.5,40.0,37.5,29.0,21020,1045,...,3.606888,16.296570,70.999405,-0.003790,1.802358,21020,CONDIS,13631192.0,-0.095702,2
4,American Airlines Group Inc.,AAL,2020,4065191,57.0,48.0,53.0,52.0,21020,1045,...,38.878998,115.990257,136.770126,-0.029870,-0.242799,21020,CONDIS,6849860.0,-0.448205,2


In [ ]:
# check the shape of the data
yearly_data.shape


(3007, 30)

In [ ]:
# only getting the 2021 data
yearly_data_2021 = yearly_data[yearly_data['year'] == 2021]

In [ ]:
## Regression
# first, add a constant in our matrix
df_spread = sm.add_constant(df_spread)
df_spread.head()

# run the regression
model = sm.OLS(df_spread['Ri-Rf'], df_spread[['const', 'Rm-Rf', 'SMB', 'HML', 'ESG']])

# get the results
results = model.fit()

## Results
print(results.summary())

NameError: name 'df_spread' is not defined

In [ ]:
yearly_data_2021.columns

Index(['CompanyName', 'ticker', 'year', 'institutionid',
       'EconomicGovernanceDimension', 'EnvironmentalDimension',
       'SPGlobalESGScore', 'SocialDimension', 'PERMNO', 'gvkey',
       'book_to_market1', 'capital_ratio1', 'enterprise_value_multiple1',
       'price_to_book1', 'return_on_equity1', 'inventory_current_asset_frac1',
       'receivable_current_asset_frac1', 'cash_ratio1', 'effective_tax_rate1',
       'debt_assets1', 'b_mkt_fama_french_3fac1', 'b_smb_fama_french_3fac1',
       'b_hml_fama_french_3fac1', 'alpha_fama_french_3fac1',
       'excess_return_fama_french_3fac1', 'permno', 'icb_industry_code',
       'mktcap', 'yearly_return', 'industry'],
      dtype='object')

In [ ]:
# run a regression 
# import statsmodels.api as sm
# With mktcap, book_to_market1, b_mkt_fama_french_3fac1, SPGlobalESGScore, and interaction between icb_industry_code and SPGlobalESGScore
# dependent variable is yearly_return

import statsmodels.api as sm
import statsmodels.formula.api as smf

# run a regression
# with mktcap, book_to_market1, b_mkt_fama_french_3fac1, SPGlobalESGScore, and interaction between icb_industry_code and SPGlobalESGScore
# dependent variable is yearly_return
model = smf.ols(formula='yearly_return ~ b_mkt_fama_french_3fac1 + mktcap + book_to_market1 + SPGlobalESGScore + icb_industry_code:SPGlobalESGScore', data=yearly_data_2021)
results = model.fit()


# output the results and export to pdf
print(results.summary())
results.summary().as_latex()








                            OLS Regression Results                            
Dep. Variable:          yearly_return   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     4.619
Date:                Tue, 06 Dec 2022   Prob (F-statistic):           3.40e-08
Time:                        23:26:04   Log-Likelihood:                -648.63
No. Observations:                1023   AIC:                             1327.
Df Residuals:                    1008   BIC:                             1401.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

'\\begin{center}\n\\begin{tabular}{lclc}\n\\toprule\n\\textbf{Dep. Variable:}                                  &  yearly\\_return  & \\textbf{  R-squared:         } &     0.060   \\\\\n\\textbf{Model:}                                          &       OLS        & \\textbf{  Adj. R-squared:    } &     0.047   \\\\\n\\textbf{Method:}                                         &  Least Squares   & \\textbf{  F-statistic:       } &     4.619   \\\\\n\\textbf{Date:}                                           & Tue, 06 Dec 2022 & \\textbf{  Prob (F-statistic):} &  3.40e-08   \\\\\n\\textbf{Time:}                                           &     23:26:04     & \\textbf{  Log-Likelihood:    } &   -648.63   \\\\\n\\textbf{No. Observations:}                               &        1023      & \\textbf{  AIC:               } &     1327.   \\\\\n\\textbf{Df Residuals:}                                   &        1008      & \\textbf{  BIC:               } &     1401.   \\\\\n\\textbf{Df Model:}          

In [ ]:
# correlation matrix
corr = yearly_data_2021.corr()
corr


,year,institutionid,EconomicGovernanceDimension,EnvironmentalDimension,SPGlobalESGScore,SocialDimension,PERMNO,gvkey,book_to_market1,capital_ratio1,...,debt_assets1,b_mkt_fama_french_3fac1,b_smb_fama_french_3fac1,b_hml_fama_french_3fac1,alpha_fama_french_3fac1,excess_return_fama_french_3fac1,permno,mktcap,yearly_return,industry
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
institutionid,NaN,1.000000,-0.109392,-0.097317,-0.126178,-0.105919,-0.160954,0.069424,-0.192981,0.033026,...,-0.172350,0.001993,-0.014498,0.019058,-0.107894,-0.006077,-0.163092,-0.027111,-0.091929,0.033858
EconomicGovernanceDimension,NaN,-0.109392,1.000000,0.789936,0.947631,0.874908,0.039614,-0.072421,-0.018654,0.089893,...,0.082176,-0.011481,-0.027052,-0.045333,0.097875,0.019361,0.038998,0.174648,0.061595,0.044167
EnvironmentalDimension,NaN,-0.097317,0.789936,1.000000,0.908678,0.823878,0.021244,-0.079478,-0.026349,0.043969,...,0.075695,-0.005590,-0.014104,-0.027889,0.054784,0.009729,0.019628,0.243138,0.038901,0.033697
SPGlobalESGScore,NaN,-0.126178,0.947631,0.908678,1.000000,0.955253,0.022313,-0.074000,-0.036655,0.076998,...,0.108053,-0.003195,-0.015596,-0.043093,0.078306,0.009186,0.021598,0.219112,0.052780,0.058015
SocialDimension,NaN,-0.105919,0.874908,0.823878,0.955253,1.000000,0.003750,-0.058830,-0.051459,0.071059,...,0.113107,-0.003496,-0.005308,-0.037753,0.073740,0.007437,0.003137,0.200116,0.049506,0.059538
PERMNO,NaN,-0.160954,0.039614,0.021244,0.022313,0.003750,1.000000,0.309406,0.032435,0.013625,...,-0.018561,0.017882,-0.011320,0.001865,0.079663,-0.013900,0.997370,-0.024296,0.037808,-0.071390
gvkey,NaN,0.069424,-0.072421,-0.079478,-0.074000,-0.058830,0.309406,1.000000,-0.008682,-0.016462,...,-0.019738,0.017421,-0.013099,0.049504,0.009454,-0.022276,0.307672,-0.039061,0.012686,0.028296
book_to_market1,NaN,-0.192981,-0.018654,-0.026349,-0.036655,-0.051459,0.032435,-0.008682,1.000000,-0.151096,...,0.122166,0.016397,0.019262,0.050196,0.156465,-0.015157,0.034194,-0.094374,0.116504,-0.160652
capital_ratio1,NaN,0.033026,0.089893,0.043969,0.076998,0.071059,0.013625,-0.016462,-0.151096,1.000000,...,0.265256,0.463599,-0.010169,-0.152095,0.243419,-0.435807,0.012990,0.001349,-0.020754,-0.032782


In [ ]:
data_complete.columns

Index(['Company Name', 'PERMNO', 'ticker', 'date', 'icb_industry_code',
       'monthly_return', 'snp_monthly_return', 'monthly_mktcap',
       'monthly_price_volume', 'gvkey', 'book_to_market', 'capital_ratio',
       'enterprise_value_multiple', 'price_to_book', 'return_on_equity',
       'inventory_current_asset_frac', 'receivable_current_asset_frac',
       'cash_ratio', 'effective_tax_rate', 'debt_assets',
       'Economic Governance Dimension', 'Environmental Dimension',
       'S&P Global ESG Score', 'Social Dimension', 'ten_yr_ytm_rf',
       'b_mkt_fama_french_3fac', 'b_smb_fama_french_3fac',
       'b_hml_fama_french_3fac', 'alpha_fama_french_3fac',
       'excess_return_fama_french_3fac', 'b_mkt_capm_mkt', 'alpha_capm',
       'excess_return_capm'],
      dtype='object')

In [ ]:
# read in data_complete.xlsx

data_complete = pd.read_excel('data_complete.xlsx')
data_analysis = pd.read_excel('Final_data_yearly.xls')
data_complete.rename(columns={'permno': 'PERMNO'}, inplace=True)
# join the b_mkt_capm_mkt from data_complete to data_analysis on PERMNO and gvkey
data_analysis_beta = data_analysis.merge(data_complete[['PERMNO', 'gvkey', 'b_mkt_capm_mkt']], on=['PERMNO', 'gvkey'], how='left')


FileNotFoundError: [Errno 2] No such file or directory: 'data_complete.xlsx'

In [ ]:
data_analysis_beta.shape
data_analysis_beta.to_excel('data_analysis_beta.xlsx')

In [ ]:
# for company with the same year and same PERMNO and gvkey, take the average of all the variables
data_complete_monthly = data_analysis_beta.groupby(['PERMNO', 'gvkey', 'date']).mean().reset_index()